In [40]:
import pandas as pd
from ydata_profiling import ProfileReport

data = pd.read_csv("../../data/structured/carbon.csv")
# profile = ProfileReport(data, title="Carbon Emissions Data Profiling Report", explorative=True)
# profile.to_file('../../data/files/analysis.html')

In [41]:
data.drop(columns=['TID', 'logger'],inplace=True)

In [42]:
def organizar_por_timestamp_context(df, col_timestamp='timestamp', col_context='context'):
    """
    Ordena o DataFrame pelo timestamp crescente e depois pelo context.
    
    Parâmetros:
        df: pandas.DataFrame
        col_timestamp: nome da coluna de timestamp
        col_context: nome da coluna de contexto
    Retorna:
        DataFrame ordenado
    """
    # Garantir que a coluna de timestamp é datetime
    df[col_timestamp] = pd.to_datetime(df[col_timestamp])
    # Ordenar pelos dois campos
    df_ordenado = df.sort_values(by=[col_context,col_timestamp], ascending=[True, True]).reset_index(drop=True)
    return df_ordenado

data = organizar_por_timestamp_context(data)


In [43]:
data['timestamp'] = pd.to_datetime(data['timestamp'])
data['ano'] = data['timestamp'].dt.year
data['mes'] = data['timestamp'].dt.month
data['dia'] = data['timestamp'].dt.day
data['hora'] = data['timestamp'].dt.hour
data['minuto'] = data['timestamp'].dt.minute
data['segundo'] = data['timestamp'].dt.second
data.drop(columns=['timestamp'], inplace=True)


In [ ]:
data

In [38]:
print(f'Total de ano diferentes: {data["ano"].nunique()}')
print(f'Total de mes diferentes: {data["mes"].nunique()}')
print(f'Total de dia diferentes: {data["dia"].nunique()}')
print(f'Total de hora diferentes: {data["hora"].nunique()}')
print(f'Total de minuto diferentes: {data["minuto"].nunique()}')
print(f'Total de segundo diferentes: {data["segundo"].nunique()}')

Total de ano diferentes: 2
Total de mes diferentes: 3
Total de dia diferentes: 31
Total de hora diferentes: 24
Total de minuto diferentes: 60
Total de segundo diferentes: 60


In [37]:
index = 0
for level in data['level']:
    if level in ['ERROR','WARN']:
        print(f'Index: {index}, Level: {level}')
        break
    index += 1
display(data.iloc[index])

Index: 8, Level: ERROR


level                                                  ERROR
context    org.wso2.carbon.crypto.provider.internal.Defau...
message    An error occurred while activating 'org.wso2.c...
ano                                                     2024
mes                                                       11
dia                                                       15
hora                                                      15
minuto                                                    44
segundo                                                   30
Name: 8, dtype: object

In [6]:
# Seleciona mensagens únicas do nível ERROR
mensagens_erro = data['message'][data['level'] == 'ERROR'].unique()

# Salva cada mensagem em uma nova linha de um arquivo .txt
with open("../../data/files/unique_error.txt", "w", encoding="utf-8") as f:
    for mensagem in mensagens_erro:
        f.write(str(mensagem) + "\n")


In [8]:
import pandas as pd

def filtrar_e_salvar_erros(
    df, 
    coluna_msg='message', 
    coluna_nivel='level', 
    termo_excluir='#getNewJsonPayload',
    nivel_erro='ERROR',
    caminho_saida='unique_error.txt'
):
    """
    Filtra mensagens que NÃO contenham o termo_excluir, reseta o índice,
    extrai mensagens únicas do nível especificado e salva em arquivo txt.
    
    Parâmetros:
    - df: DataFrame original.
    - coluna_msg: nome da coluna da mensagem.
    - coluna_nivel: nome da coluna do nível.
    - termo_excluir: termo a ser removido das mensagens.
    - nivel_erro: nível de erro a ser filtrado ('ERROR', por padrão).
    - caminho_saida: caminho para salvar o arquivo .txt.
    
    Retorna:
    - DataFrame filtrado.
    """
    # Remove linhas com o termo indesejado
    dataM = df[~df[coluna_msg].str.contains(termo_excluir, na=False)].reset_index(drop=True)
    
    # Seleciona mensagens únicas do nível especificado
    mensagens_erro = dataM[coluna_msg][dataM[coluna_nivel] == nivel_erro].unique()
    
    # Salva no arquivo txt
    with open(caminho_saida, "w", encoding="utf-8") as f:
        for mensagem in mensagens_erro:
            f.write(str(mensagem) + "\n")
    
    return dataM.reset_index(drop=True)

# Exemplo de uso:
dataM = filtrar_e_salvar_erros(data, caminho_saida="../../data/files/unique_error1.txt")


In [13]:
dataCNHR = filtrar_e_salvar_erros(dataM,caminho_saida="../../data/files/unique_error2.txt",termo_excluir='Could not handle request')

2024-11-15 15:44:30.720 2025-01-22 23:59:51.188


In [16]:
data

,timestamp,level,context,message
0,2024-11-15 15:44:30.720,INFO,org.wso2.carbon.core.internal.CarbonCoreActivator,Starting WSO2 Carbon...
1,2024-11-15 15:44:30.739,INFO,org.wso2.carbon.core.internal.CarbonCoreActivator,"Operating System : Linux 5.4.0-198-generic, amd64"
2,2024-11-15 15:44:30.739,INFO,org.wso2.carbon.core.internal.CarbonCoreActivator,Java Home : /opt/java/openjdk
3,2024-11-15 15:44:30.740,INFO,org.wso2.carbon.core.internal.CarbonCoreActivator,Java Version : 11.0.10
4,2024-11-15 15:44:30.740,INFO,org.wso2.carbon.core.internal.CarbonCoreActivator,Java VM : OpenJDK 64-Bit Server VM 11...
...,...,...,...,...
1051828,2025-01-22 23:59:45.668,WARN,org.wso2.carbon.databridge.agent.endpoint.Data...,"No receiver is reachable at reconnection, will..."
1051829,2025-01-22 23:59:45.674,INFO,org.wso2.carbon.databridge.core.DataBridge,user adminsuper@nosi.cv connected
1051830,2025-01-22 23:59:45.676,ERROR,org.wso2.carbon.databridge.agent.endpoint.Data...,Error while trying to connect to the endpoint....
1051831,2025-01-22 23:59:45.676,ERROR,org.wso2.carbon.databridge.agent.endpoint.Data...,Error while trying to connect to the endpoint....


In [44]:
data.to_csv("../../data/structured/carbon_feature_engineered.csv", index=False)